# Web scrapping para la funcion de fitness de los tiempos de los pasajeros


In [22]:
#!pip install selenium
#!pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.0 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\el_su\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Parte 1: Llegadas

En esta primera parte extraeremos los tiempos desde la puerta de la terminal correspondiente. Existen varias puertas de salida, por lo que se elegirá la que menos tiempo de tránsito dé.

In [1]:
import re
# Constantes

pagina_web_base = "https://www.aena.es/es/map.html"
search_box_name = "search"
puertas_hacia_exterior = ["Exterior terminal (junto puerta 1)", "Exterior terminal (junto puerta 3)", "Exterior planta facturación"]
gate_regexp = r'Puertas? [A-Za-z][0-9](\d|\n)?'
salas_salida = ["Sala 1", "Sala 2", "Sala 6", "Sala 10"]


In [2]:
# variables comunes

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

import time

driver = webdriver.Firefox()

driver.get(pagina_web_base)

-- Recuperacion de las puertas existentes


In [3]:
driver.refresh()

#poner puerta en el buscador
start_input = driver.find_element(by="name", value=search_box_name)
start_input.send_keys("Puerta")
start_input.send_keys(Keys.RETURN)

# recoger todas las puertas
wait = WebDriverWait(driver, 4)
puertas = []
list_of_puertas_ext = wait.until(EC.visibility_of_element_located((By.ID, "pois-list")))


for puerta in list_of_puertas_ext.find_elements(by=By.TAG_NAME,value= "li"):
    if re.match(gate_regexp, puerta.text):
        puerta_sin_terminal = puerta.text.split("\n")[0]
        puertas.append(puerta_sin_terminal)


In [5]:
print(puertas)
print(len(puertas))

['Puerta A10', 'Puerta A11', 'Puerta A12', 'Puerta A13', 'Puerta A14', 'Puerta A9', 'Puerta A16', 'Puertas A1 y A2', 'Puertas A3 y A4', 'Puertas A5 y A6', 'Puertas A7 y A8', 'Puertas B18-B20', 'Puertas B21-B25', 'Puertas B26-B29', 'Puertas B30 y B31', 'Puertas C34-C38', 'Puertas C39-C42', 'Puerta C148', 'Puerta C149', 'Puerta C150', 'Puerta D161', 'Puerta D162', 'Puerta D163', 'Puertas C43 y C44', 'Puertas C45 y C46', 'Puertas C47 y C48', 'Puertas C49 y C50', 'Puertas D53 y D54', 'Puertas D55 y D56', 'Puertas D57 y D58', 'Puertas D59 y D60', 'Puertas D61 y D62', 'Puertas D63 y D64', 'Puertas D65 y D66', 'Puerta F90', 'Puerta F91', 'Puerta F92', 'Puerta F93', 'Puerta E69', 'Puerta E74', 'Puerta E75', 'Puerta E78', 'Puerta E79', 'Puerta E80', 'Puerta E81', 'Puertas E67 y E68', 'Puertas E70 y E71', 'Puertas E72 y E73', 'Puertas E76 y E77', 'Puerta E82', 'Puerta E83', 'Puertas H1 y H3', 'Puertas H13 y H15', 'Puertas H16 y H18', 'Puertas H19 y H21', 'Puertas H2 y H4', 'Puertas H22 y H24', '

Ahora tenemos las puertas y los puntos a los que queremos llegar, asi que vamos a extraer el tiempo de punto de interes a puerta. Lo haremos para cada puerta y cogeremos el menor tiempo de todos los posibles.


Estructura: 
{
    "Puerta1": { "Salida3" : 45 },
    "Puerta2": { "Salida3" : 41 },
    "Puerta3": { "Salida1" : 3 }
}

In [6]:
puertas_salidas = [ [(puerta, ext) for ext in puertas_hacia_exterior] for puerta in puertas]
salas_salidas_tuples = [ [(puerta, sala) for sala in salas_salida] for puerta in puertas]

In [7]:
print(salas_salidas_tuples)

[[('Puerta A10', 'Sala 1'), ('Puerta A10', 'Sala 2'), ('Puerta A10', 'Sala 6'), ('Puerta A10', 'Sala 10')], [('Puerta A11', 'Sala 1'), ('Puerta A11', 'Sala 2'), ('Puerta A11', 'Sala 6'), ('Puerta A11', 'Sala 10')], [('Puerta A12', 'Sala 1'), ('Puerta A12', 'Sala 2'), ('Puerta A12', 'Sala 6'), ('Puerta A12', 'Sala 10')], [('Puerta A13', 'Sala 1'), ('Puerta A13', 'Sala 2'), ('Puerta A13', 'Sala 6'), ('Puerta A13', 'Sala 10')], [('Puerta A14', 'Sala 1'), ('Puerta A14', 'Sala 2'), ('Puerta A14', 'Sala 6'), ('Puerta A14', 'Sala 10')], [('Puerta A9', 'Sala 1'), ('Puerta A9', 'Sala 2'), ('Puerta A9', 'Sala 6'), ('Puerta A9', 'Sala 10')], [('Puerta A16', 'Sala 1'), ('Puerta A16', 'Sala 2'), ('Puerta A16', 'Sala 6'), ('Puerta A16', 'Sala 10')], [('Puertas A1 y A2', 'Sala 1'), ('Puertas A1 y A2', 'Sala 2'), ('Puertas A1 y A2', 'Sala 6'), ('Puertas A1 y A2', 'Sala 10')], [('Puertas A3 y A4', 'Sala 1'), ('Puertas A3 y A4', 'Sala 2'), ('Puertas A3 y A4', 'Sala 6'), ('Puertas A3 y A4', 'Sala 10')], 

In [13]:
tiempos = {}
for index, lista_puerta in enumerate(puertas_salidas):
    posibles_tiempos = []
    print(f"puerta {index+1} de {len(puertas)}")

    
    for puerta, ext in lista_puerta:
        driver.refresh()
        wait = WebDriverWait(driver, 4)
        start_input = wait.until(EC.visibility_of_element_located((By.NAME, search_box_name)))
        start_input.clear()
        start_input.send_keys(puerta)
        
        wait = WebDriverWait(driver, 4)
        list_res = wait.until(EC.visibility_of_element_located((By.ID, "pois-list")))
        list_res.find_element(by=By.TAG_NAME, value="li").click()
        wait = WebDriverWait(driver, 4)
        btn_como_llegar = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "btn")))
        btn_como_llegar.click()

        # ahora vamos punto por punto de informacion
        origen = wait.until(EC.visibility_of_element_located((By.NAME, search_box_name)))

        origen.clear()
        origen.send_keys(ext)
        list_res = wait.until(EC.visibility_of_element_located((By.ID, "pois-list")))
        list_res.find_element(by=By.TAG_NAME, value="li").click()
        time.sleep(1)
        info = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "info")))
        info_raw = info.text
        tiempo = info_raw.split("\n")[1].split(" ")[0]
        
        if tiempo == "1":
            time.sleep(8)
            info = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "info")))
            info_raw = info.text
            tiempo = info_raw.split("\n")[1].split(" ")[0]
        try:  
            if " h." in info_raw or " h " in info_raw or "horas" in info_raw:
                tiempo = int(tiempo) * 60 
                aux = info_raw.split("\n")[1].split(" ")
                tiempo += int(aux[2])
            posibles_tiempos.append(int(tiempo))
        except:
            posibles_tiempos.append(1000)
        print(f"Puerta: {puerta} Origen: {ext} - Tiempo: {tiempo}")
            
    tiempo_minimo = min(posibles_tiempos)
    index_minimo = posibles_tiempos.index(tiempo_minimo)
    temp = {puerta : {"origen" : puertas_hacia_exterior[index_minimo], "tiempo" : tiempo_minimo}}
    print(f"Puerta: {puerta} - Origen: {puertas_hacia_exterior[index_minimo]} - Tiempo: {tiempo_minimo} minutos")
    print("-------------------------------------------------- \n")
    tiempos.update(temp)
        

    
    

puerta 1 de 123
Puerta: Puerta A10 Origen: Exterior terminal (junto puerta 1) - Tiempo: 23
Puerta: Puerta A10 Origen: Exterior terminal (junto puerta 3) - Tiempo: 22
Puerta: Puerta A10 Origen: Exterior planta facturación - Tiempo: 44
Puerta: Puerta A10 - Origen: Exterior terminal (junto puerta 3) - Tiempo: 22 minutos
-------------------------------------------------- 

puerta 2 de 123
Puerta: Puerta A11 Origen: Exterior terminal (junto puerta 1) - Tiempo: 23
Puerta: Puerta A11 Origen: Exterior terminal (junto puerta 3) - Tiempo: 22
Puerta: Puerta A11 Origen: Exterior planta facturación - Tiempo: 44
Puerta: Puerta A11 - Origen: Exterior terminal (junto puerta 3) - Tiempo: 22 minutos
-------------------------------------------------- 

puerta 3 de 123
Puerta: Puerta A12 Origen: Exterior terminal (junto puerta 1) - Tiempo: 23
Puerta: Puerta A12 Origen: Exterior terminal (junto puerta 3) - Tiempo: 22
Puerta: Puerta A12 Origen: Exterior planta facturación - Tiempo: 44
Puerta: Puerta A12 - O

In [14]:
print(tiempos)

{'Puerta A10': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 22}, 'Puerta A11': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 22}, 'Puerta A12': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 22}, 'Puerta A13': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 21}, 'Puerta A14': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 21}, 'Puerta A9': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 22}, 'Puerta A16': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 20}, 'Puertas A1 y A2': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 23}, 'Puertas A3 y A4': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 22}, 'Puertas A5 y A6': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 22}, 'Puertas A7 y A8': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 21}, 'Puertas B18-B20': {'origen': 'Exterior terminal (junto puerta 3)', 'tiempo': 17}, 'Puertas B21-B25': {'origen': 'Exterior

In [15]:

# Convert the dictionary to a DataFrame
df_tiempos = pd.DataFrame.from_dict(tiempos, orient='index')


# Save the DataFrame to an Excel file
df_tiempos.to_excel('tiempos_ida.xlsx', index_label='Puerta')


In [16]:
df_tiempos

,origen,tiempo
Puerta A10,Exterior terminal (junto puerta 3),22
Puerta A11,Exterior terminal (junto puerta 3),22
Puerta A12,Exterior terminal (junto puerta 3),22
Puerta A13,Exterior terminal (junto puerta 3),21
Puerta A14,Exterior terminal (junto puerta 3),21
...,...,...
Puertas S47 y S48,Exterior planta facturación,37
Puertas S49 y S50,Exterior planta facturación,37
Puertas S5 y S6,Exterior planta facturación,36
Puertas S7 y S8,Exterior planta facturación,36


# Parte 2: De la puerta al exterior


In [9]:
tiempos = pd.DataFrame(columns=["Origen", "Destino", "Tiempo"])
for index, lista_puerta in enumerate(salas_salidas_tuples):
    posibles_tiempos = []
    print(f"puerta {index+1} de {len(puertas)}")

    
    for puerta, punto_salida in lista_puerta:
        driver.refresh()
        wait = WebDriverWait(driver, 4)
        start_input = wait.until(EC.visibility_of_element_located((By.NAME, search_box_name)))
        start_input.clear()
        start_input.send_keys(punto_salida)
        
        wait = WebDriverWait(driver, 4)
        list_res = wait.until(EC.visibility_of_element_located((By.ID, "pois-list")))
        posibles_salas = list_res.find_elements(by=By.TAG_NAME, value="li")
        
        element = [elem for elem in posibles_salas if elem.text.startswith(punto_salida)][0]
        element.click()


        wait = WebDriverWait(driver, 4)
        btn_como_llegar = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "btn")))
        btn_como_llegar.click()

        # ahora vamos punto por punto de informacion
        origen = wait.until(EC.visibility_of_element_located((By.NAME, search_box_name)))

        origen.clear()
        origen.send_keys(puerta)
        list_res = wait.until(EC.visibility_of_element_located((By.ID, "pois-list")))
        list_res.find_elements(by=By.TAG_NAME, value="li")[0].click()
        time.sleep(1)
        info = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "info")))
        info_raw = info.text
        tiempo = info_raw.split("\n")[1].split(" ")[0]
        if tiempo == "1":
            time.sleep(8)
            info = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "info")))
            info_raw = info.text
            tiempo = info_raw.split("\n")[1].split(" ")[0]
        try:
            if " h " in info_raw or "1 h" in info_raw or "horas" in info_raw:
                tiempo = int(tiempo) * 60 
                aux = info_raw.split("\n")[1].split(" ")
                if str.isdigit(aux[2]):
                    tiempo += int(aux[2])
            posibles_tiempos.append(int(tiempo))
        except:
            posibles_tiempos.append(1000)
        print(f"Origen: {puerta} Salida: {punto_salida} - Tiempo: {tiempo}")

    tiempo_minimo = min(posibles_tiempos)
    index_minimo = posibles_tiempos.index(tiempo_minimo)
    tiempos.loc[len(tiempos)] = [puerta, salas_salida[index_minimo], tiempo_minimo]

    print(f"Origen: {puerta} - Destino: {salas_salida[index_minimo]} - Tiempo: {tiempo_minimo} minutos")
    print("-------------------------------------------------- \n")
        

    
    

puerta 1 de 123
Origen: Puerta A10 Salida: Sala 1 - Tiempo: 19
Origen: Puerta A10 Salida: Sala 2 - Tiempo: 27
Origen: Puerta A10 Salida: Sala 6 - Tiempo: 35
Origen: Puerta A10 Salida: Sala 10 - Tiempo: 48
Origen: Puerta A10 - Destino: Sala 1 - Tiempo: 19 minutos
-------------------------------------------------- 

puerta 2 de 123
Origen: Puerta A11 Salida: Sala 1 - Tiempo: 18
Origen: Puerta A11 Salida: Sala 2 - Tiempo: 26
Origen: Puerta A11 Salida: Sala 6 - Tiempo: 35
Origen: Puerta A11 Salida: Sala 10 - Tiempo: 48
Origen: Puerta A11 - Destino: Sala 1 - Tiempo: 18 minutos
-------------------------------------------------- 

puerta 3 de 123
Origen: Puerta A12 Salida: Sala 1 - Tiempo: 18
Origen: Puerta A12 Salida: Sala 2 - Tiempo: 26
Origen: Puerta A12 Salida: Sala 6 - Tiempo: 34
Origen: Puerta A12 Salida: Sala 10 - Tiempo: 47
Origen: Puerta A12 - Destino: Sala 1 - Tiempo: 18 minutos
-------------------------------------------------- 

puerta 4 de 123
Origen: Puerta A13 Salida: Sala 1 - 

In [10]:
tiempos

,Origen,Destino,Tiempo
0,Puerta A10,Sala 1,19
1,Puerta A11,Sala 1,18
2,Puerta A12,Sala 1,18
3,Puerta A13,Sala 1,18
4,Puerta A14,Sala 1,18
...,...,...,...
118,Puertas S47 y S48,Sala 10,30
119,Puertas S49 y S50,Sala 10,31
120,Puertas S5 y S6,Sala 10,31
121,Puertas S7 y S8,Sala 10,31


In [11]:



# Save the DataFrame to an Excel file
tiempos.to_excel('tiempos_vuelta_a_salas_comunes.xlsx', index_label='Puerta')


In [12]:
driver.quit()